In [15]:
import numpy as np 
import pandas as pd

In [16]:
# 忽略警告信息
import warnings
warnings.filterwarnings("ignore")

## 数据集的准备

In [17]:
from sklearn.model_selection import train_test_split
train=pd.read_csv('datas/house_data.csv')
y=train['SalePrice']
train1=train.drop(['Id','SalePrice'],axis=1)
X=pd.get_dummies(train1).reset_index(drop=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)

In [18]:
# tmp=train.isnull().sum()
# tmp[tmp>0] 

#### 模型测评

In [19]:
from sklearn.metrics import mean_squared_error
def benchmark2(model,testset,label):
    pred=model.predict(testset)
    if pred[pred<0].shape[0]>0:
        print('Neg Value')
    rmse=np.sqrt(mean_squared_error(label,pred))
    lrmse=np.sqrt(mean_squared_error(np.log(label),np.log(pred)))

    print('RMSE:',rmse)
    print('LRMSE:',lrmse)
    return lrmse

## 基础模型训练

### ElasticNet

In [20]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

In [21]:
kfolds=KFold(n_splits=10, shuffle=True, random_state=123)
e_l1ratio=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.85,0.9,0.95]
e_alphas=np.logspace(-10,2.8,150)

In [22]:
def elastic_train_test(alpha,l1ratio):
    e_model=make_pipeline(RobustScaler(),ElasticNetCV(alphas=[alpha],l1_ratio=[l1ratio]))
    e_model.fit(X_train,y_train)
    lrmse=benchmark2(e_model,X_test,y_test)
    return lrmse

In [23]:
elastic_train_test(50,0.5) 

RMSE: 64803.88956616406
LRMSE: 0.3056812482960621


0.3056812482960621

In [ ]:
elastic_model=make_pipeline(RobustScaler(),ElasticNetCV(alphas=e_alphas,l1_ratio=e_l1ratio)).fit(X_train,y_train)

In [ ]:
benchmark2(elastic_model,X_test,y_test)

In [ ]:
elastic_model.steps[1][1].alpha_

In [ ]:
elastic_model.steps[1][1].l1_ratio_

### XGBoost训练

In [ ]:
import xgboost as xgb
xg_reg=xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.7,learning_rate=0.01,max_depth=3,n_estimators=3400,subsample=0.7,nthread=6,seed=123)

In [ ]:
xg_reg.fit(X_train,y_train)

In [ ]:
benchmark2(xg_reg,X_test,y_test)

<br><br>

## Stacking集成算法

### 底层算法

In [3]:
from mlxtend.regressor import StackingCVRegressor

In [184]:
alphas_alt=np.logspace(-10,2.8,150)

In [13]:
ridge=make_pipeline(RobustScaler(),RidgeCV(alphas=alphas_alt,cv=kfolds))

NameError: name 'alphas_alt' is not defined

In [14]:
lasso=make_pipeline(RobustScaler(),LassoCV(alphas=alphas_alt,cv=kfolds))

NameError: name 'alphas_alt' is not defined

In [189]:
elasticnet=make_pipeline(RobustScaler(),ElasticNetCV(alphas=e_alphas,cv=kfolds, l1_ratio=e_l1ratio))

In [190]:
xgboost=make_pipeline(RobustScaler(),xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.7,learning_rate=0.01,max_depth=3,n_estimators=3460,subsample=0.7,reg_alpha=0.00006,gamma=0,nthread=6,scale_pos_weight=1,seed=27))

### 上层算法

In [12]:
# 使用原训练集中的feature
stack_alg=StackingCVRegressor(regressors=(ridge,lasso,elasticnet,xgboost),
                              meta_regressor=xgboost, use_features_in_secondary=True)

NameError: name 'ridge' is not defined

In [193]:
stackX=np.array(X_train)

In [194]:
stacky=np.array(y_train)

In [196]:
stack_alg.fit(stackX,stacky)

StackingCVRegressor(cv=5,
          meta_regressor=Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('xgbregressor', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_de...    reg_alpha=6e-05, reg_lambda=1, scale_pos_weight=1, seed=27,
       silent=True, subsample=0.7))]),
          refit=True,
          regressors=(Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('ridgecv', RidgeCV(alphas=array([1.00000e-10, 1.21873e-10, ..., 5.17719e+02, 6.30957e+02]),
    cv=KFold(n_splits=10, random_state=123...  reg_alpha=6e-05, reg_lambda=1, scale_pos_weight=1, seed=27,
       silent=True, subsample=0.7))])),
          shuffle=True, store_train_meta_features=False,
          use_features

In [199]:
p

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [200]:
benchmark2(stack_alg,X_test,y_test)

RMSE: 22251.381642116565
LRMSE: 0.10578249817750969


0.10578249817750969